In [25]:
import arcpy
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [26]:
# Paths to your data
ml_folder = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\RASTERS_ALIGNED"

# Output directories
OUT_DIR = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\OUTPUTS"
os.makedirs(OUT_DIR, exist_ok=True)

In [27]:
# Load training points data
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True

training_points_fc = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb\TrainingPts"
# --- Read Training Points Data into Pandas DataFrame ---
fields = [f.name for f in arcpy.ListFields(training_points_fc)]
# Assume label field is named "Label" and coordinates are "x", "y"
skip_fields = ["ObjectID", "Shape", "Shape_Length", "Shape_Area"]  # Adjust this based on your dataset

# Extract features and labels
rows = []
with arcpy.da.SearchCursor(training_points_fc, fields) as cursor:
    for row in cursor:
        data = {fields[i]: row[i] for i in range(len(row)) if fields[i] not in skip_fields}
        rows.append(data)

# Convert to DataFrame
df = pd.DataFrame(rows)
print(f"[Info] Training data: {df.shape[0]} rows and {df.shape[1]} columns")

# Separate features (X) and labels (y)
y = df['Label'].values  # Label column assumed to be 'Label'
X = df.drop(columns=['Label'])  # Drop the label column from the features

[Info] Training data: 409 rows and 50 columns


In [28]:
# --- Define Preprocessing Pipelines ---
# Column names
cont_cols = [col for col in X.columns if X[col].dtype in ['float64', 'int64']]  # Continuous columns
lith_cols = [col for col in X.columns if 'lith_' in col]  # Lithology one-hot encoded columns

# Preprocessing pipeline for the SVM
svm_prep = ColumnTransformer([
    ("cont_pipe", Pipeline([
        ("impute", SimpleImputer(strategy="median")),
        ("scale", StandardScaler())
    ]), cont_cols),
    ("lith_impute", SimpleImputer(strategy="most_frequent"), lith_cols)
], remainder="drop")

# --- Combine All Features (Lithology, Geochemical, Continuous) ---
feature_order = cont_cols + lith_cols  # All features we want to include

In [29]:
# --- SVM Model Definition ---
def build_svm(C=5.0):
    """Build SVM pipeline with RBF kernel, scale continuous features, and balanced classes."""
    return Pipeline([
        ("prep", svm_prep),  # Apply preprocessing
        ("svc", SVC(kernel="rbf", C=C, gamma="scale", class_weight="balanced", probability=True, random_state=42))
    ])

In [30]:
import arcpy

# Define paths for the fishnet and training points
gdb = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb"
training_points_fc = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb\TrainingPts"
fishnet_fc = os.path.join(gdb, "Fishnet_5km")  # Path to the fishnet
fishnet_cell_size = 5000  # Define the cell size (e.g., 5000 meters)

# Step 1: Create the fishnet over your AOI
# Get the extent of the AOI (or use the extent of the training points)
extent = arcpy.Describe(training_points_fc).extent
origin = f"{extent.XMin} {extent.YMin}"
y_axis = f"{extent.XMin} {extent.YMin + 10}"  # Small offset to define Y axis direction
opposite_corner = f"{extent.XMax} {extent.YMax}"

# Create fishnet
if arcpy.Exists(fishnet_fc):
    arcpy.management.Delete(fishnet_fc)  # Delete if it already exists

arcpy.management.CreateFishnet(
    out_feature_class=fishnet_fc,
    origin_coord=origin,
    y_axis_coord=y_axis,
    cell_width=fishnet_cell_size,
    cell_height=fishnet_cell_size,
    number_rows=0,  # Automatically calculate the number of rows based on extent
    number_columns=0,  # Automatically calculate the number of columns based on extent
    corner_coord=opposite_corner,
    labels="NO_LABELS",
    template="#",
    geometry_type="POLYGON"
)

print(f"Fishnet created: {fishnet_fc}")

# Step 2: Add a block_id field to the fishnet (if not already exists)
if "block_id" not in [f.name for f in arcpy.ListFields(fishnet_fc)]:
    arcpy.management.AddField(fishnet_fc, "block_id", "LONG")

# Step 3: Populate the block_id field with the OID (Object ID) from the fishnet
oid_field = arcpy.Describe(fishnet_fc).OIDFieldName
arcpy.management.CalculateField(fishnet_fc, "block_id", f"!{oid_field}!", "PYTHON3")

# Step 4: Spatially join the training points with the fishnet
# This will assign a 'block_id' to each point in your training data
output_fc = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb\TrainingPts_with_block_id"

arcpy.analysis.SpatialJoin(
    target_features=training_points_fc,
    join_features=fishnet_fc,
    out_feature_class=output_fc,
    join_type="KEEP_COMMON",  # Only keep points that intersect with fishnet
    match_option="INTERSECT"
)

print(f"Spatial join complete: {output_fc}")

# Step 5: Add block_id to the DataFrame (assuming you are using pandas)
# Read the resulting feature class into a pandas DataFrame
fields = [f.name for f in arcpy.ListFields(output_fc)]
rows = []
with arcpy.da.SearchCursor(output_fc, fields) as cursor:
    for row in cursor:
        rows.append(row)

# Create DataFrame from rows
df_with_block_id = pd.DataFrame(rows, columns=fields)
print(f"DataFrame with block_id: {df_with_block_id.shape[0]} rows and {df_with_block_id.shape[1]} columns")

# Now you can use `df_with_block_id` for your machine learning tasks


Fishnet created: C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb\Fishnet_5km
Spatial join complete: C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb\TrainingPts_with_block_id
DataFrame with block_id: 409 rows and 54 columns


In [31]:
# Load the dataset that already contains 'block_id'
training_points_with_block_id_fc = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\MPM_LIRHANDA_CORRIDOR\MPM_LIRHANDA_CORRIDOR.gdb\TrainingPts_with_block_id"

# Convert the feature class to a DataFrame
fields = [f.name for f in arcpy.ListFields(training_points_with_block_id_fc)]
rows = []
with arcpy.da.SearchCursor(training_points_with_block_id_fc, fields) as cursor:
    for row in cursor:
        rows.append(row)

df_with_block_id = pd.DataFrame(rows, columns=fields)

# Check the columns to ensure 'block_id' is present
print(f"Columns: {df_with_block_id.columns}")
print(f"Number of rows: {df_with_block_id.shape[0]}")

# Separate features (X) and labels (y)
y = df_with_block_id['Label'].values  # Assuming 'Label' is the column for the target variable
X = df_with_block_id.drop(columns=['Label', 'block_id'])  # Drop 'Label' and 'block_id' from features

# Define continuous and lithology columns (modify based on your dataset)
cont_cols = [col for col in X.columns if X[col].dtype in ['float64', 'int64']]  # Continuous columns
lith_cols = [col for col in X.columns if 'lith_' in col]  # Lithology one-hot encoded columns

# Preprocessing pipeline for the SVM
svm_prep = ColumnTransformer([
    ("cont_pipe", Pipeline([
        ("impute", SimpleImputer(strategy="median")),
        ("scale", StandardScaler())
    ]), cont_cols),
    ("lith_impute", SimpleImputer(strategy="most_frequent"), lith_cols)
], remainder="drop")

# SVM model pipeline
def build_svm(C=5.0):
    return Pipeline([
        ("prep", svm_prep),  # Apply preprocessing
        ("svc", SVC(kernel="rbf", C=C, gamma="scale", class_weight="balanced", probability=True, random_state=42))
    ])

# --- Cross-validation: GroupKFold ---
C_grid = [1, 2, 5, 10]  # Hyperparameter tuning grid for C
best_C, best_pr, oof_best = None, -1.0, None
gkf = GroupKFold(n_splits=5)

# GroupKFold cross-validation
for C in C_grid:
    oof = np.zeros(len(y))  # Out-of-Fold predictions
    
    for tr, te in gkf.split(X, y, df_with_block_id['block_id']):  # Use 'block_id' for grouping
        model = build_svm(C).fit(X.iloc[tr], y[tr])
        oof[te] = model.predict_proba(X.iloc[te])[:, 1]  # Probability for class 1
    
    # Evaluate using PR-AUC and ROC-AUC
    pr = average_precision_score(y, oof)
    roc = roc_auc_score(y, oof)
    
    print(f"C={C:>2}  OOF ROC-AUC={roc:.3f} | OOF PR-AUC={pr:.3f}")
    if pr > best_pr:
        best_pr, best_C, oof_best = pr, C, oof

print(f"Chosen C (by OOF PR-AUC): {best_C}")


Columns: Index(['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'Label', 'TxtLabel',
       'Ag', 'As', 'Bi', 'ClayAIOH', 'Cu', 'Curvature_Gen10', 'Curv_Plan_10',
       'Curv_Profile_10', 'dem10', 'dem10_1', 'distfaults2', 'FebyMn',
       'Ferrous', 'ironoxide', 'KbyAl', 'KbyNa', 'KbyRb', 'KbyTi', 'OM', 'Pb',
       'PH', 'Sb', 'Slope_10', 'Tl', 'UbyK', 'Zn', 'Lith_And_11',
       'Lith_BIF_15', 'Lith_Bpl_17', 'Lith_Bsl_13', 'Lith_Bs_43', 'Lith_Cgl_5',
       'Lith_Dac_12', 'Lith_Dd_28', 'Lith_Gc_21', 'Lith_Gl_20', 'Lith_Gnt_40',
       'Lith_Gn_18', 'Lith_Grt_4', 'Lith_Kv_41', 'Lith_Mud_3', 'Lith_Ny_42',
       'Lith_Phy_39', 'Lith_Rhy_10', 'Lith_Sy_23', 'Lith_Tuf_14', 'Lith_Tv_31',
       'block_id'],
      dtype='object')
Number of rows: 409
C= 1  OOF ROC-AUC=0.979 | OOF PR-AUC=0.893
C= 2  OOF ROC-AUC=0.981 | OOF PR-AUC=0.893
C= 5  OOF ROC-AUC=0.982 | OOF PR-AUC=0.909
C=10  OOF ROC-AUC=0.975 | OOF PR-AUC=0.904
Chosen C (by OOF PR-AUC): 5


In [32]:
# --- Threshold Selection: PR Curve (F2-optimal) ---
prec, rec, thr = precision_recall_curve(y, oof_best)
beta = 2  # F2 score focuses on recall
f2 = (1 + beta**2) * (prec * rec) / ((beta**2) * prec + rec + 1e-9)
k = f2.argmax()
THRESH_SVM = float(thr[max(k-1, 0)])  # Optimal threshold from PR curve

print(f"SVM OOF threshold (F2-opt): {THRESH_SVM:.3f}  |  P={prec[k]:.3f}  R={rec[k]:.3f}")
print(f"OOF ROC-AUC: {roc_auc_score(y, oof_best):.3f} | OOF PR-AUC: {average_precision_score(y, oof_best):.3f}")

SVM OOF threshold (F2-opt): 0.082  |  P=0.702  R=0.971
OOF ROC-AUC: 0.982 | OOF PR-AUC: 0.909


In [33]:
# Ensure you're using the DataFrame that contains 'block_id'
# If you used `TrainingPts_with_block_id`, use that DataFrame here.
df = df_with_block_id  # Make sure this is the DataFrame with the 'block_id'

# --- Spatial Hold-out Evaluation ---
gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=456)

# Use 'block_id' from the correct DataFrame (df_with_block_id)
tr, te = next(gss.split(X, y, df['block_id']))  # Ensure you're using df with block_id

svm_hold = build_svm(best_C).fit(X.iloc[tr], y[tr])
proba_te = svm_hold.predict_proba(X.iloc[te])[:, 1]  # Test set probabilities

print("\n=== SVM: Spatial hold-out (25% blocks) ===")
print(f"ROC-AUC: {roc_auc_score(y[te], proba_te):.3f} | PR-AUC: {average_precision_score(y[te], proba_te):.3f}")
pred_te = (proba_te >= THRESH_SVM).astype(int)
print(classification_report(y[te], pred_te, digits=3))



=== SVM: Spatial hold-out (25% blocks) ===
ROC-AUC: 0.986 | PR-AUC: 0.951
              precision    recall  f1-score   support

           0      0.976     0.932     0.953        88
           1      0.769     0.909     0.833        22

    accuracy                          0.927       110
   macro avg      0.873     0.920     0.893       110
weighted avg      0.935     0.927     0.929       110



In [34]:
# --- Final Model Training on All Data ---
svm = build_svm(best_C).fit(X, y)
print("\nSVM model is ready for deployment. Variables available: `svm` and `THRESH_SVM`.")


SVM model is ready for deployment. Variables available: `svm` and `THRESH_SVM`.


In [38]:
import os
import numpy as np
import pandas as pd
import arcpy
from arcpy.sa import *

# --- ArcGIS setup ---
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

# ====== CONFIG / INPUTS ======
try:
    OUT_DIR
except NameError:
    OUT_DIR = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\OUTPUTS"
os.makedirs(OUT_DIR, exist_ok=True)

ml_folder = r"C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\RASTERS_ALIGNED"
OUT_PROB  = os.path.join(OUT_DIR, "Prospectivity_SVM.tif")
OUT_BIN   = os.path.join(OUT_DIR, "Targets_SVM_binary.tif")

# ====== HELPERS ======
def norm_name(s: str) -> str:
    stem = os.path.splitext(os.path.basename(s))[0]
    return ''.join(ch for ch in stem.lower() if ch.isalnum() or ch == '_')

def read_block(raster_path: str, lower_left_point: arcpy.Point, w: int, h: int) -> np.ndarray:
    ras = arcpy.Raster(raster_path)
    arr = arcpy.RasterToNumPyArray(ras, lower_left_point, w, h).astype("float32")
    nd = ras.noDataValue
    if nd is not None:
        arr[arr == nd] = np.nan
    return arr

def safe_save_tiff(ras, out_path, pixel_type="32_BIT_FLOAT", nodata=None):
    if arcpy.Exists(out_path):
        arcpy.management.Delete(out_path)
    try:
        ras.save(out_path)
        return
    except Exception as e:
        print(f"[WARN] Direct save failed for {os.path.basename(out_path)} -> {e}")
        scratch_gdb = arcpy.env.scratchGDB
        tmp_name = arcpy.CreateUniqueName("tmp_save", scratch_gdb)
        ras.save(tmp_name)
        if arcpy.Exists(out_path):
            arcpy.management.Delete(out_path)
        arcpy.management.CopyRaster(
            in_raster=tmp_name,
            out_rasterdataset=out_path,
            pixel_type=pixel_type,
            nodata_value=nodata,
            format="TIFF"
        )
        arcpy.management.Delete(tmp_name)

# ====== MODEL-EXPECTED COLUMNS ======
# Prefer the exact columns the pipeline was fit on.
if hasattr(svm, "feature_names_in_"):
    required_cols = list(svm.feature_names_in_)
else:
    required_cols = list(feature_order)  # fall back if needed

print(f"[INFO] Model expects {len(required_cols)} columns.")

# ====== BUILD LOOKUP FROM FILES ======
tif_files = [f for f in os.listdir(ml_folder) if f.lower().endswith(".tif")]
feat_lookup = {norm_name(f): os.path.join(ml_folder, f) for f in tif_files}

# Split expected columns into those with rasters (“band features”) vs. missing (admin/others)
band_features = [c for c in required_cols if norm_name(c) in feat_lookup]
missing_feats = [c for c in required_cols if norm_name(c) not in feat_lookup]

if missing_feats:
    print("[WARN] These expected columns have no raster on disk; will be filled with zeros:")
    print("       " + ", ".join(missing_feats))

# ====== REFERENCE RASTER ======
ref_raster_path = os.path.join(ml_folder, "dem10.tif")
if not arcpy.Exists(ref_raster_path):
    raise FileNotFoundError(f"Reference raster not found: {ref_raster_path}")

ref_raster = arcpy.Raster(ref_raster_path)
ncols = int(ref_raster.width)
nrows = int(ref_raster.height)
cell_w = ref_raster.meanCellWidth
cell_h = ref_raster.meanCellHeight
xmin  = ref_raster.extent.XMin
ymin  = ref_raster.extent.YMin

nodata_out = -9999.0
out_prob = np.full((nrows, ncols), np.nan, dtype="float32")

# ====== BLOCKED PREDICTION ======
BLOCK = 1024

for r0 in range(0, nrows, BLOCK):
    rh = min(BLOCK, nrows - r0)
    for c0 in range(0, ncols, BLOCK):
        cw = min(BLOCK, ncols - c0)
        ll_win = arcpy.Point(xmin + c0 * cell_w, ymin + r0 * cell_h)

        # Read only the band-backed features
        band_list = []
        for f in band_features:
            f_path = feat_lookup[norm_name(f)]
            band_list.append(read_block(f_path, ll_win, cw, rh))
        if band_list:
            stack = np.stack(band_list, axis=0)  # [n_band_features, rh, cw]
            X_block = pd.DataFrame(
                stack.reshape(len(band_features), -1).T,
                columns=band_features
            )
        else:
            # No band features? create empty frame with correct length
            X_block = pd.DataFrame(index=np.arange(rh*cw))

        # Fill missing columns (admin etc.) with zeros, then reorder to model expectation
        for col in missing_feats:
            X_block[col] = 0.0
        # Make sure all required columns exist
        missing_now = [c for c in required_cols if c not in X_block.columns]
        for col in missing_now:
            X_block[col] = 0.0
        X_block = X_block[required_cols].fillna(0.0)

        # Predict proba for class 1 and reshape to the tile
        proba = svm.predict_proba(X_block)[:, 1].astype("float32").reshape(rh, cw)

        out_prob[r0:r0 + rh, c0:c0 + cw] = proba
        print(f"Processed rows {r0}-{r0+rh} cols {c0}-{c0+cw}", end="\r")

print("\n[INFO] Finished probability prediction.")

# ====== THRESHOLD + SAVE ======
threshold = 0.5
out_bin = np.zeros_like(out_prob, dtype="uint8")
valid = ~np.isnan(out_prob)
out_bin[valid & (out_prob >= threshold)] = 1

ll_ref = arcpy.Point(xmin, ymin)
prob_ras = arcpy.NumPyArrayToRaster(out_prob, ll_ref, cell_w, cell_h, value_to_nodata=nodata_out)
bin_ras  = arcpy.NumPyArrayToRaster(out_bin,  ll_ref, cell_w, cell_h, value_to_nodata=None)

safe_save_tiff(prob_ras, OUT_PROB, pixel_type="32_BIT_FLOAT", nodata=nodata_out)
safe_save_tiff(bin_ras,  OUT_BIN,  pixel_type="8_BIT_UNSIGNED", nodata=None)

print(f"[OK] Saved probability raster: {OUT_PROB}")
print(f"[OK] Saved binary raster:      {OUT_BIN}")


[INFO] Model expects 52 columns.
[WARN] These expected columns have no raster on disk; will be filled with zeros:
       OBJECTID, Shape, Join_Count, TARGET_FID, TxtLabel
Processed rows 8192-8938 cols 8192-9031
[INFO] Finished probability prediction.
[OK] Saved probability raster: C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\OUTPUTS\Prospectivity_SVM.tif
[OK] Saved binary raster:      C:\Users\USER\Desktop\PROJECTS\ESRI\DATA\OUTPUTS\Targets_SVM_binary.tif


In [40]:
# Save the probability raster output
to_save = np.where(np.isfinite(out_prob), out_prob, nodata_out)
rast_p = arcpy.NumPyArrayToRaster(to_save, ref_raster.extent.lowerLeft, ref_raster.meanCellWidth, 
                                   ref_raster.meanCellHeight, nodata_out)
rast_p.save(OUT_PROB)

# Save binary targets based on threshold
bin_arr = np.zeros_like(to_save, dtype="uint8")
valid_prob = np.isfinite(out_prob)
bin_arr[valid_prob] = (out_prob[valid_prob] >= THRESH_SVM).astype("uint8")
nodata_bin = 255  # For NoData values
bin_arr[~valid_prob] = nodata_bin
rast_b = arcpy.NumPyArrayToRaster(bin_arr, ref_raster.extent.lowerLeft, ref_raster.meanCellWidth, 
                                  ref_raster.meanCellHeight, nodata_bin)
rast_b.save(OUT_BIN)


# Build pyramids for better visualization
arcpy.management.CalculateStatistics(OUT_PROB)
arcpy.management.BuildPyramids(OUT_PROB)
arcpy.management.CalculateStatistics(OUT_BIN)
arcpy.management.BuildPyramids(OUT_BIN)

print("✅ All steps completed successfully.")

✅ All steps completed successfully.
